In [1]:
# this is the pipeline of phenotype analysis
import sys
import os
import pandas as pd
import json
sys.path.append('..')
import abd_profile

In [2]:
sig_dir = '../result/immu/SE_sig'
se_dir = '../result/immu/SE'
outdir = '../result/immu/combine'
if not os.path.exists(outdir):
    os.makedirs(outdir)
sig_sp_dict = json.load(open(os.path.join(sig_dir,'cluster_sp.json')))
se_sp_dict = json.load(open(os.path.join(se_dir,'cluster_sp.json')))
sig_sp_dict['sig1'] = sig_sp_dict['sig1'].split(',')
sig_sp_dict['sig2'] = sig_sp_dict['sig2'].split(',')
se_sp_dict['supercluster_S3'] = se_sp_dict['supercluster_S3'].split(',')
se_sp_dict['cluster_S1-C20'] = se_sp_dict['cluster_S1-C20'].split(',')

# load num of sig
sig_sps = json.load(open(os.path.join(sig_dir,'existed_sp.json')))
se_sps = json.load(open(os.path.join(se_dir,'existed_sp.json')))


In [3]:
print(set(sig_sp_dict['sig1']).intersection(set(se_sp_dict['supercluster_S3'])))
print(set(sig_sp_dict['sig2']).intersection(set(se_sp_dict['supercluster_S3'])))
print(set(sig_sp_dict['sig1']).intersection(set(se_sp_dict['cluster_S1-C20'])))
print(len(set(sig_sp_dict['sig2']).intersection(set(se_sp_dict['cluster_S1-C20']))))

set()
set()
set()
6


In [4]:
new_sp_dict = {}
new_sp_dict['pos'] = list(set(sig_sp_dict['sig2']).union(set(se_sp_dict['cluster_S1-C20'])))
new_sp_dict['neg'] = list(set(sig_sp_dict['sig1']).union(set(se_sp_dict['supercluster_S3'])))

sample_sps = {'pos':{}, 'neg':{}}
# neg
for sample in sig_sps['sig1'].keys():
    sp_list_sig = sig_sps['sig1'][sample].split(',')
    sp_list_se = se_sps['supercluster_S3'][sample].split(',')
    all_sp = list(set(sp_list_sig).union(set(sp_list_se)))
    sample_sps['neg'][sample] = all_sp

# pos 
for sample in sig_sps['sig2'].keys():
    sp_list_sig = sig_sps['sig2'][sample].split(',')
    sp_list_se = se_sps['cluster_S1-C20'][sample].split(',')
    all_sp = list(set(sp_list_sig).union(set(sp_list_se)))
    sample_sps['pos'][sample] = all_sp


In [5]:

score_df = pd.DataFrame(index= sample_sps['pos'].keys(), columns=['score'])
for sample in sample_sps['pos'].keys():
    pos = len(set(sample_sps['pos'][sample]).intersection(set(new_sp_dict['pos'])))/len(new_sp_dict['pos'])
    neg = len(set(sample_sps['neg'][sample]).intersection(set(new_sp_dict['neg'])))/len(new_sp_dict['neg'])
    score_df.loc[sample, 'score'] = (pos - neg +1)/2
score_df.to_csv(os.path.join(outdir, 'score_disc.tsv'), sep='\t')

In [6]:
# Cohort OS Sample_id TOPOB01 OS12
DS1 = pd.read_csv('../data/immu/DS1_oncology_clinical_data.csv', header=0)
DS1 = DS1[DS1['Cohort']=='Disc']
DS1.set_index('Sample_id', inplace=True)
DS1.dropna(subset=['OS12'], inplace=True)
DS1['Sample_id'] = DS1.index

pred_disc = pd.read_csv(os.path.join(outdir, 'score_disc.tsv'), sep='\t', header=0, index_col=0)
DS1['TOPOB01'] = pred_disc.loc[DS1.index, 'score']
DS1.to_csv(os.path.join(outdir, 'pred_disc.tsv'), sep='\t', index=False)
DS1.loc[DS1[DS1['TOPOB01'] < 0.499].index, 'TOPOB01'] = 0
DS1.loc[DS1[DS1['TOPOB01'] >= 0.643].index, 'TOPOB01'] = 1
pred_DS1 = DS1[(DS1['TOPOB01'] == 0) | (DS1['TOPOB01'] == 1)]
pred_DS1.to_csv(os.path.join(outdir, 'pred_binary_disc.tsv'), sep='\t', index=False)
print(len(pred_DS1[((pred_DS1['OS12']=='NR')&(pred_DS1['TOPOB01'] == 0))])/len(pred_DS1[pred_DS1['TOPOB01'] == 0]), len(pred_DS1[((pred_DS1['OS12']=='NR')&(pred_DS1['TOPOB01'] == 0))]), len(pred_DS1[pred_DS1['TOPOB01'] == 0]))
print(len(pred_DS1[((pred_DS1['OS12']=='NR')&(pred_DS1['TOPOB01'] == 1))])/len(pred_DS1[pred_DS1['TOPOB01'] == 1]), len(pred_DS1[((pred_DS1['OS12']=='NR')&(pred_DS1['TOPOB01'] == 1))]), len(pred_DS1[pred_DS1['TOPOB01'] == 1]))

# NR [64] < 0.50 < gray_zone [88] < 0.64 < R [78]
# auc = 0.680

# add akk
normal_n = 0
abnormal_n = 0
zero_n = 0
for sample in DS1.index:
    if DS1.loc[sample, 'TOPOB01'] < 1 and DS1.loc[sample, 'TOPOB01'] > 0:
        if DS1.loc[sample, 'AKK_TRICHO'] == 'Low':
            DS1.loc[sample, 'TOPOB01'] = 1
            normal_n += 1
        else:
            DS1.loc[sample, 'TOPOB01'] = 0
            if DS1.loc[sample, 'AKK_TRICHO'] == 'Zero':
                zero_n += 1
            abnormal_n += 1
DS1.to_csv(os.path.join(outdir, 'pred_disc_akk.tsv'), sep='\t', index=False)
print(normal_n, abnormal_n, normal_n/(normal_n+abnormal_n))
print(normal_n/230, abnormal_n/230)
print(zero_n/230, (abnormal_n-zero_n)/230)
print(len(DS1[((DS1['OS12']=='NR')&(DS1['TOPOB01'] == 0))])/len(DS1[DS1['TOPOB01'] == 0]))
print(len(DS1[((DS1['OS12']=='NR')&(DS1['TOPOB01'] == 1))])/len(DS1[DS1['TOPOB01'] == 1]))
print(len(DS1[DS1['TOPOB01'] == 0]))
print(len(DS1[DS1['TOPOB01'] == 1]))
# auc = 0.61

0.65625 42 64
0.2948717948717949 23 78
28 60 0.3181818181818182
0.12173913043478261 0.2608695652173913
0.21304347826086956 0.04782608695652174
0.5887096774193549
0.36792452830188677
124
106


In [7]:
for c,  slist in new_sp_dict.items():
    print(c, len(slist))

pos 32
neg 36


In [8]:
# time points
time_df = pd.read_csv('../data/immu/DS5_longitudinal_clinical_data.csv', header=0)
time_df.dropna(how='any', inplace=True)

In [9]:
time_df[(time_df['Sample_id_V0'].isin(pred_disc.index)) & (time_df['Sample_id_V3'].isin(pred_disc.index))]

,Patient_id,Sample_id_V0,AKK_TRICHO_V0,Sample_id_V3,AKK_TRICHO_V3


In [10]:
sorted(pred_disc.index)

['C26490',
 'C26505',
 'LUM2_001',
 'LUM2_002',
 'LUM2_003',
 'LUM2_004',
 'LUM2_005',
 'LUM2_006',
 'LUM2_007',
 'LUM2_008',
 'LUM2_009',
 'LUM2_010',
 'LUM2_011',
 'LUM2_013',
 'LUM2_015',
 'LUM2_017',
 'LUM2_018',
 'LUM2_019',
 'LUM2_020',
 'LUM2_021',
 'LUM2_022',
 'LUM2_023',
 'LUM2_024',
 'LUM2_025',
 'LUM2_026',
 'LUM2_027',
 'LUM2_028',
 'LUM2_029',
 'LUM2_030',
 'LUM2_031',
 'LUM2_032',
 'LUM2_033',
 'LUM2_034',
 'LUM2_035',
 'LUM2_036',
 'LUM2_037',
 'LUM2_038',
 'LUM2_039',
 'LUM2_040',
 'LUM2_041',
 'LUM2_042',
 'LUM2_043',
 'LUM2_044',
 'LUM2_046',
 'LUM2_047',
 'LUM2_049',
 'LUM2_050',
 'LUM2_051',
 'LUM2_052',
 'LUM2_053',
 'LUM2_054',
 'LUM2_055',
 'LUM2_056',
 'LUM2_057',
 'LUM2_058',
 'LUM2_059',
 'LUM2_060',
 'LUM2_061',
 'LUM2_062',
 'LUM2_063',
 'LUM2_064',
 'LUM2_065',
 'LUM2_066',
 'LUM2_067',
 'LUM2_068',
 'LUM2_069',
 'LUM2_070',
 'LUM2_071',
 'LUM2_072',
 'LUM2_073',
 'LUM2_074',
 'LUM2_075',
 'LUM2_076',
 'LUM2_077',
 'LUM2_078',
 'LUM2_079',
 'LUM2_080',
 'L

In [11]:
time_df[(time_df['Sample_id_V0'].isin(pred_disc.index))]

,Patient_id,Sample_id_V0,AKK_TRICHO_V0,Sample_id_V3,AKK_TRICHO_V3
0,10039,LUM_216,Low,LUM3_380,Low
1,10040,LUM_217,Zero,LUM3_373,Zero
3,10046,LUM_251,Low,LUM3_386,High
8,10055,LUM_339,Low,LUM3_166,High
9,10061,LUM_262,Zero,LUM3_390,Zero
10,10062,LUM2_004,Zero,LUM3_060,Zero
12,10084,LUM2_040,Low,LUM3_087,Low
20,10143,LUM3_236,Zero,LUM3_335,Zero
23,10152,LUM3_340,Low,LUM3_367,Low
35,20013,LUM_104,Zero,LUM3_371,Zero


In [12]:
time_df[(time_df['Sample_id_V3'].isin(pred_disc.index))]

,Patient_id,Sample_id_V0,AKK_TRICHO_V0,Sample_id_V3,AKK_TRICHO_V3


In [13]:
set(sorted(time_df['Sample_id_V3'])).intersection(sorted(pred_disc.index))

set()

In [14]:
abd_path = '../data/immu/merged_species.txt'
d_df = pd.read_csv('../data/sp_d.tsv', sep='\t', header=0, index_col=0)
raw_profile = abd_profile.input_profile(abd_path, transfer=True)
raw_profile = raw_profile[list(set(d_df.index).intersection(set(raw_profile.columns)))]

In [15]:
raw_profile.index

Index(['LUM3_251', 'LUM3_252', 'LUM3_253', 'LUM3_254', 'LUM3_255', 'LUM3_256',
       'LUM3_257', 'LUM3_258', 'IMM_008', 'IMM_158',
       ...
       'LUM3_242', 'LUM3_243', 'LUM3_244', 'LUM3_245', 'LUM3_246', 'LUM3_247',
       'LUM3_248', 'IMM_156', 'LUM3_249', 'LUM3_250'],
      dtype='object', length=499)

In [16]:
time_df[(time_df['Sample_id_V0'].isin(pred_disc.index)) & (time_df['Sample_id_V3'].isin(raw_profile.index))]

,Patient_id,Sample_id_V0,AKK_TRICHO_V0,Sample_id_V3,AKK_TRICHO_V3


In [17]:
time_df[time_df['Sample_id_V3'].isin(raw_profile.index)]

,Patient_id,Sample_id_V0,AKK_TRICHO_V0,Sample_id_V3,AKK_TRICHO_V3


In [18]:
tmp_list = []
for x in raw_profile.index:
    if x.startswith('LUM3_'):
        tmp_list.append(x)

In [19]:
sorted(time_df['Sample_id_V3'])

['LUM3_060',
 'LUM3_063',
 'LUM3_087',
 'LUM3_107',
 'LUM3_115',
 'LUM3_122',
 'LUM3_166',
 'LUM3_214',
 'LUM3_231',
 'LUM3_320',
 'LUM3_335',
 'LUM3_345',
 'LUM3_354',
 'LUM3_367',
 'LUM3_371',
 'LUM3_373',
 'LUM3_380',
 'LUM3_386',
 'LUM3_388',
 'LUM3_390',
 'LUM3_480',
 'LUM3_518',
 'LUM3_521',
 'LUM3_522',
 'LUM3_528',
 'LUM3_529',
 'LUM3_633',
 'LUM3_640',
 'LUM3_642',
 'LUM3_714',
 'LUM3_934',
 'LUM3_941']

In [20]:
set(tmp_list).intersection(set(time_df['Sample_id_V3']))

set()

In [21]:
tmp_list21 = []
tmp_list22 = []
for idx in raw_profile.index:
    if idx.startswith('LUM3_'):
        prev = idx.replace('LUM3_', 'LUM_')
        if prev in raw_profile.index:
            print(prev, idx)
            tmp_list21.append(prev)
            tmp_list22.append(idx)

LUM_251 LUM3_251
LUM_252 LUM3_252
LUM_253 LUM3_253
LUM_254 LUM3_254
LUM_255 LUM3_255
LUM_256 LUM3_256
LUM_257 LUM3_257
LUM_258 LUM3_258
LUM_259 LUM3_259
LUM_260 LUM3_260
LUM_261 LUM3_261
LUM_262 LUM3_262
LUM_263 LUM3_263
LUM_264 LUM3_264
LUM_265 LUM3_265
LUM_266 LUM3_266
LUM_267 LUM3_267
LUM_268 LUM3_268
LUM_269 LUM3_269
LUM_270 LUM3_270
LUM_271 LUM3_271
LUM_274 LUM3_274
LUM_275 LUM3_275
LUM_278 LUM3_278
LUM_282 LUM3_282
LUM_283 LUM3_283
LUM_120 LUM3_120
LUM_125 LUM3_125
LUM_221 LUM3_221
LUM_341 LUM3_341
LUM_101 LUM3_101
LUM_249 LUM3_249
LUM_250 LUM3_250


In [23]:
len(tmp_list21)

33

In [26]:
DS1 = pd.read_csv('../data/immu/DS1_oncology_clinical_data.csv', header=0)
taken_df = DS1[DS1['Sample_id'].isin(tmp_list21+tmp_list22)].dropna(subset=['OS12'])

In [30]:
set(tmp_list21+tmp_list22).difference(set(taken_df['Sample_id']))

{'LUM3_278'}

In [31]:
tmp_list22.remove('LUM3_278')

In [32]:
tmp_list21.remove('LUM_278')

In [34]:
len(tmp_list21)

32

In [36]:
','.join(tmp_list21)

'LUM_251,LUM_252,LUM_253,LUM_254,LUM_255,LUM_256,LUM_257,LUM_258,LUM_259,LUM_260,LUM_261,LUM_262,LUM_263,LUM_264,LUM_265,LUM_266,LUM_267,LUM_268,LUM_269,LUM_270,LUM_271,LUM_274,LUM_275,LUM_282,LUM_283,LUM_120,LUM_125,LUM_221,LUM_341,LUM_101,LUM_249,LUM_250'